<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re

In [2]:
Data_basic = pd.DataFrame()

In [3]:
Data_basic = pd.DataFrame(columns=['GameID','season','types','week','HomeTeam_ID','AwayTeam_ID','Winner'])
week = list(range(1, 19))
seasons=list(range(2017,2023))

In [4]:
seasons

[2017, 2018, 2019, 2020, 2021, 2022]

In [5]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"

  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [6]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [7]:
Data_basic


,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,400951566,2017,2,1,17,12,Away
1,400951567,2017,2,1,2,20,Home
2,400951570,2017,2,1,3,1,Away
3,400951572,2017,2,1,4,33,Away
4,400951574,2017,2,1,5,23,Away
...,...,...,...,...,...,...,...
1564,401437957,2022,2,18,21,19,Home
1565,401437962,2022,2,18,25,22,Home
1566,401437963,2022,2,18,26,14,Home
1567,401437959,2022,2,18,28,6,Home


In [8]:
type(Data_basic['HomeTeam_ID'][0])





str

In [38]:
GameID='400951566'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])

In [10]:
drive_df['result']


0              TD
1          FUMBLE
2           DOWNS
3              TD
4              FG
5            PUNT
6              TD
7            PUNT
8            PUNT
9              TD
10    END OF HALF
11           PUNT
12           PUNT
13             TD
14           PUNT
15           PUNT
16             TD
17           PUNT
18             FG
19             TD
20          DOWNS
21           PUNT
22           PUNT
23             TD
24           PUNT
Name: result, dtype: object

In [11]:
for i in drive_df:
  print(i)

$ref
id
description
sequenceNumber
yards
isScore
offensivePlays
result
shortDisplayResult
displayResult
team.$ref
endTeam.$ref
start.period.type
start.period.number
start.clock.value
start.clock.displayValue
start.yardLine
start.text
end.period.type
end.period.number
end.clock.value
end.clock.displayValue
end.yardLine
end.text
timeElapsed.value
timeElapsed.displayValue
source.id
source.description
plays.$ref
plays.count
plays.pageIndex
plays.pageSize
plays.pageCount
plays.items


In [12]:
Hometeam=Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']

In [13]:
Hometeam[0]

'17'

In [19]:
yards_Home=0
yards_Away=0
TD_Home=0
TD_Away=0
PUNT_Home=0
PUNT_Away=0
FG_Home=0
FG_Away=0
Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']
#if re.search(r"teams/(\d+)", drive_df['team.$ref'][0]).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']:
for index, row in drive_df.iterrows():
    if re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'][0]:
      yards_Home=yards_Home + row['yards']
      if row['result']=='TD':
        TD_Home=TD_Home+1
      elif row['result']=='PUNT':
        PUNT_Home=PUNT_Home+1
      elif row['result']=='FG':
        FG_Home=FG_Home+1
      



    else:
      yards_Away=yards_Away + row['yards'] 
      if row['result']=='TD':
        TD_Away=TD_Away+1
      elif row['result']=='PUNT':
        PUNT_Away=PUNT_Away+1
      elif row['result']=='FG':
        FG_Away=FG_Away+1
print(FG_Home)
print(FG_Away)
print(PUNT_Home)      
print(PUNT_Away)



2
0
5
6


In [49]:
drive_df['plays.items'][3][2]['type']['text']

'Pass Reception'

In [63]:
homescore=0
awayScore=0
for plays in range(len(drive_df['plays.items'])):
  for detail in range(len(drive_df['plays.items'][plays])):
    print(drive_df['plays.items'][plays][detail]['type']['text'])
    print(drive_df['plays.items'][plays][detail]['statYardage'])
    #print(drive_df['plays.items'][plays][detail]['scoreValue'])
    print(drive_df['plays.items'][plays][detail]['awayScore'])
    print(drive_df['plays.items'][plays][detail]['homeScore'])

Kickoff
26
0
0
Pass Incompletion
0
0
0
Pass Reception
8
0
0
Rush
8
0
0
Rush
3
0
0
Pass Reception
19
0
0
Rush
5
0
0
Pass Reception
16
0
0
Pass Incompletion
12
0
0
Rush
0
0
0
Pass Incompletion
0
0
0
Rushing Touchdown
2
0
7
Kickoff
0
0
7
Fumble Recovery (Opponent)
7
0
7
Pass Incompletion
0
0
7
Pass Reception
3
0
7
Rush
10
0
7
Pass Incompletion
0
0
7
Rush
-1
0
7
Pass Incompletion
5
0
13
Pass Reception
5
0
7
Rush
0
0
7
Rush
9
0
7
Rush
4
0
7
Pass Reception
14
0
7
Pass Reception
8
0
7
Pass Reception
5
0
7
Rush
9
0
7
Pass Reception
3
0
7
Pass Reception
11
0
7
Pass Reception
18
0
7
Pass Incompletion
0
0
7
Rush
2
0
7
Passing Touchdown
7
7
7
Kickoff
0
7
7
Rush
3
7
7
Pass Reception
27
7
7
Pass Reception
15
7
7
Rush
13
7
7
Rush
-5
7
7
Pass Incompletion
0
7
7
Rush
4
7
7
Rush
0
7
7
End Period
0
7
7
Field Goal Good
25
7
10
Kickoff
31
7
10
Pass Incompletion
0
7
10
Rush
4
7
10
Sack
-3
7
10
Punt
0
7
10
Rush
4
7
10
Rush
4
7
10
Pass Reception
22
7
10
Pass Incompletion
0
7
10
Rush
0
7
10
Pass Reception
16
7

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
plays.items,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...,[{'$ref': 'http://sports.core.api.espn.com/v2/...


In [39]:
drive_df

,$ref,id,description,sequenceNumber,yards,isScore,offensivePlays,result,shortDisplayResult,displayResult,...,timeElapsed.value,timeElapsed.displayValue,source.id,source.description,plays.$ref,plays.count,plays.pageIndex,plays.pageSize,plays.pageCount,plays.items
0,http://sports.core.api.espn.com/v2/sports/foot...,4009515661,"9 plays, 73 yards, 2:52",1,73,True,9,TD,TD,Touchdown,...,172.0,2:52,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
1,http://sports.core.api.espn.com/v2/sports/foot...,4009515662,"1 play, 7 yards, 0:08",2,7,False,1,FUMBLE,FUMBLE,Fumble,...,8.0,0:08,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,2,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
2,http://sports.core.api.espn.com/v2/sports/foot...,4009515663,"7 plays, 22 yards, 2:35",3,22,False,7,DOWNS,DOWNS,Downs,...,155.0,2:35,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,8,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
3,http://sports.core.api.espn.com/v2/sports/foot...,4009515664,"12 plays, 90 yards, 6:22",4,90,True,12,TD,TD,Touchdown,...,382.0,6:22,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
4,http://sports.core.api.espn.com/v2/sports/foot...,4009515665,"8 plays, 67 yards, 3:07",5,67,True,8,FG,FG,Field Goal,...,187.0,3:07,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,11,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
5,http://sports.core.api.espn.com/v2/sports/foot...,4009515666,"3 plays, 1 yard, 1:39",6,1,False,3,PUNT,PUNT,Punt,...,99.0,1:39,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,5,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
6,http://sports.core.api.espn.com/v2/sports/foot...,4009515667,"11 plays, 82 yards, 5:50",7,82,True,11,TD,TD,Touchdown,...,350.0,5:50,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
7,http://sports.core.api.espn.com/v2/sports/foot...,4009515668,"5 plays, 6 yards, 3:43",8,6,False,5,PUNT,PUNT,Punt,...,223.0,3:43,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,7,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
8,http://sports.core.api.espn.com/v2/sports/foot...,4009515669,"3 plays, 4 yards, 0:57",9,4,False,3,PUNT,PUNT,Punt,...,57.0,0:57,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,4,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
9,http://sports.core.api.espn.com/v2/sports/foot...,40095156610,"12 plays, 92 yards, 2:34",10,92,True,12,TD,TD,Touchdown,...,154.0,2:34,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,18,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
